In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
%cd gdrive/My Drive

In [ ]:
import os

if not os.path.isdir('neuroSAT'):
  !git clone --recurse-submodules https://github.com/dmeoli/neuroSAT
  %cd neuroSAT
else:
  %cd neuroSAT
  !git pull

In [ ]:
!pip install -r requirements.txt

In [25]:
datasets = {'uniform-random-3-sat': {'train': ['uf50-218', 'uuf50-218',
                                               'uf100-430', 'uuf100-430'],
                                     'val': ['uf50-218', 'uuf50-218',
                                             'uf100-430', 'uuf100-430'],
                                     'inner_test': ['uf50-218', 'uuf50-218',
                                                    'uf100-430', 'uuf100-430'],
                                     'test': ['uf250-1065', 'uuf250-1065'],
                                     'kt_test': ['flat30-60',
                                                 'flat50-115',
                                                 'flat75-180',
                                                 'flat100-239',
                                                 'flat125-301',
                                                 'flat150-360',
                                                 'flat175-417',
                                                 'flat200-479']},
            'graph-coloring': {'train': ['flat50-115'],
                               'val': ['flat50-115'],
                               'inner_test': ['flat50-115'],
                               'test': ['flat30-60',
                                        'flat75-180',
                                        'flat100-239',
                                        'flat125-301',
                                        'flat150-360',
                                        'flat175-417',
                                        'flat200-479']}}

# GraphQSAT

In [ ]:
%cd GraphQSAT

### Build C++

In [ ]:
%cd minisat
!sudo ln -s --force /usr/local/lib/python3.7/dist-packages/numpy/core/include/numpy /usr/include/numpy  # https://stackoverflow.com/a/44935933/5555994
!make distclean && CXXFLAGS=-w make && make python-wrap PYTHON=python3.7
!apt install swig
!swig -fastdispatch -c++ -python minisat/gym/GymSolver.i
%cd ..

## Uniform Random 3-SAT

We split *(u)uf50-218* and *(u)uf100-430* into three subsets: 800 training problems, 100 validation, and 100 test problems.

For generalization experiments, we use 100 problems from all the other benchmarks.

To evaluate the knowledge transfer properties of the trained models across different task families, we use 100 problems from all the *graph coloring* benchmarks.

In [14]:
PROBLEM_TYPE='uniform-random-3-sat'

In [ ]:
!bash train_val_test_split.sh "$PROBLEM_TYPE"

### Add metadata for evaluation (train and validation set)

In [ ]:
for TRAIN_PROBLEM_NAME, VAL_PROBLEM_NAME in zip(datasets[PROBLEM_TYPE]['train'],
                                                datasets[PROBLEM_TYPE]['val']):
  !python add_metadata.py --eval-problems-paths ../data/"$PROBLEM_TYPE"/train/"$TRAIN_PROBLEM_NAME"
  !python add_metadata.py --eval-problems-paths ../data/"$PROBLEM_TYPE"/val/"$VAL_PROBLEM_NAME"

### Train

In [ ]:
for TRAIN_PROBLEM_NAME, VAL_PROBLEM_NAME in zip(datasets[PROBLEM_TYPE]['train'],
                                                datasets[PROBLEM_TYPE]['val']):
  !python dqn.py \
    --logdir log \
    --env-name sat-v0 \
    --train-problems-paths ../data/"$PROBLEM_TYPE"/train/"$TRAIN_PROBLEM_NAME" \
    --eval-problems-paths ../data/"$PROBLEM_TYPE"/val/"$VAL_PROBLEM_NAME" \
    --lr 0.00002 \
    --bsize 64 \
    --buffer-size 20000 \
    --eps-init 1.0 \
    --eps-final 0.01 \
    --gamma 0.99 \
    --eps-decay-steps 30000 \
    --batch-updates 50000 \
    --history-len 1 \
    --init-exploration-steps 5000 \
    --step-freq 4 \
    --target-update-freq 10 \
    --loss mse \
    --opt adam \
    --save-freq 500 \
    --grad_clip 1 \
    --grad_clip_norm_type 2 \
    --eval-freq 1000 \
    --eval-time-limit 3600 \
    --core-steps 4 \
    --expert-exploration-prob 0.0 \
    --priority_alpha 0.5 \
    --priority_beta 0.5 \
    --e2v-aggregator sum \
    --n_hidden 1 \
    --hidden_size 64 \
    --decoder_v_out_size 32 \
    --decoder_e_out_size 1 \
    --decoder_g_out_size 1 \
    --encoder_v_out_size 32 \
    --encoder_e_out_size 32 \
    --encoder_g_out_size 32 \
    --core_v_out_size 64 \
    --core_e_out_size 64 \
    --core_g_out_size 32 \
    --activation relu \
    --penalty_size 0.1 \
    --train_time_max_decisions_allowed 500 \
    --test_time_max_decisions_allowed 500 \
    --no_max_cap_fill_buffer \
    --lr_scheduler_gamma 1 \
    --lr_scheduler_frequency 3000 \
    --independent_block_layers 0

### Add metadata for evaluation (test set)

In [ ]:
for PROBLEM_NAME in datasets[PROBLEM_TYPE]['inner_test']:
  !python add_metadata.py --eval-problems-paths ../data/"$PROBLEM_TYPE"/test/"$PROBLEM_NAME"

for PROBLEM_NAME in datasets[PROBLEM_TYPE]['test'] + datasets[PROBLEM_TYPE]['kt_test']:
  !python add_metadata.py --eval-problems-paths ../data/"$PROBLEM_TYPE"/"$PROBLEM_NAME"

### Evaluate

In [4]:
models = {'uf50-218': ('Aug04_16-28-54_98b46997a24d', 
                       'model_50001'),
          'uuf50-218': ('Sep12_10-11-09_4730bf8f8ad0', 
                        'model_50006'),
          'uf100-430': ('Aug04_21-40-43_98b46997a24d', 
                        'model_50061'),
          'uuf100-430': ('Sep12_16-06-06_4730bf8f8ad0', 
                         'model_45085')}

We test these trained models on the inner test sets.

In [ ]:
for PROBLEM_NAME in datasets[PROBLEM_TYPE]['inner_test']:
  for MODEL_DECISION in [10, 50, 100, 300, 500, 1000]:
    MODEL_DIR = models[PROBLEM_NAME][0]
    CHECKPOINT = models[PROBLEM_NAME][1]
    !python evaluate.py \
      --logdir log \
      --env-name sat-v0 \
      --core-steps -1 \
      --eps-final 0.0 \
      --eval-time-limit 100000000000000 \
      --no_restarts \
      --test_time_max_decisions_allowed "$MODEL_DECISION" \
      --eval-problems-paths ../data/"$PROBLEM_TYPE"/test/"$PROBLEM_NAME" \
      --model-dir runs/"$MODEL_DIR" \
      --model-checkpoint "$CHECKPOINT".chkp \
      >> runs/"$MODEL_DIR"/"$PROBLEM_NAME"-graphqsat-max"$MODEL_DECISION".tsv

We test the trained models on the outer test sets.

In [5]:
for SAT_MODEL in models.keys():
  for PROBLEM_NAME in datasets[PROBLEM_TYPE]['test']:
    for MODEL_DECISION in [10, 50, 100, 300, 500, 1000]:
      MODEL_DIR = models[SAT_MODEL][0]
      CHECKPOINT = models[SAT_MODEL][1]
      !python evaluate.py \
        --logdir log \
        --env-name sat-v0 \
        --core-steps -1 \
        --eps-final 0.0 \
        --eval-time-limit 100000000000000 \
        --no_restarts \
        --test_time_max_decisions_allowed "$MODEL_DECISION" \
        --eval-problems-paths ../data/"$PROBLEM_TYPE"/"$PROBLEM_NAME" \
        --model-dir runs/"$MODEL_DIR" \
        --model-checkpoint "$CHECKPOINT".chkp \
        >> runs/"$MODEL_DIR"/"$PROBLEM_NAME"-graphqsat-max"$MODEL_DECISION".tsv

We test the trained models on the *graph coloring* test sets.

In [ ]:
for SAT_MODEL in models.keys():
  for PROBLEM_NAME in datasets[PROBLEM_TYPE]['kt_test']:
    for MODEL_DECISION in [10, 50, 100, 300, 500, 1000]:
      MODEL_DIR = models[SAT_MODEL][0]
      CHECKPOINT = models[SAT_MODEL][1]
      !python evaluate.py \
        --logdir log \
        --env-name sat-v0 \
        --core-steps -1 \
        --eps-final 0.0 \
        --eval-time-limit 100000000000000 \
        --no_restarts \
        --test_time_max_decisions_allowed "$MODEL_DECISION" \
        --eval-problems-paths ../data/graph-coloring/"$PROBLEM_NAME" \
        --model-dir runs/"$MODEL_DIR" \
        --model-checkpoint "$CHECKPOINT".chkp \
        >> runs/"$MODEL_DIR"/"$PROBLEM_NAME"-graphqsat-max"$MODEL_DECISION".tsv

In [ ]:
%cd ..

/content/gdrive/My Drive/neuro-sat


## Graph Coloring

Graph coloring benchmarks have only 100 problems each, except for *flat50-115* which contains 1000, so we split it into three subsets: 800 training problems, 100 validation, and 100 test problems.

For generalization experiments, we use 100 problems from all the other benchmarks.

In [7]:
PROBLEM_TYPE='graph-coloring'

In [ ]:
!bash train_val_test_split.sh "$PROBLEM_TYPE"

### Add metadata for evaluation (train and validation set)

In [ ]:
for TRAIN_PROBLEM_NAME, VAL_PROBLEM_NAME in zip(datasets[PROBLEM_TYPE]['train'],
                                                datasets[PROBLEM_TYPE]['val']):
  !python add_metadata.py --eval-problems-paths ../data/"$PROBLEM_TYPE"/train/"$TRAIN_PROBLEM_NAME"
  !python add_metadata.py --eval-problems-paths ../data/"$PROBLEM_TYPE"/val/"$VAL_PROBLEM_NAME"

### Train

In [ ]:
for TRAIN_PROBLEM_NAME, VAL_PROBLEM_NAME in zip(datasets[PROBLEM_TYPE]['train'],
                                                datasets[PROBLEM_TYPE]['val']):
  !python dqn.py \
    --logdir log \
    --env-name sat-v0 \
    --train-problems-paths ../data/"$PROBLEM_TYPE"/train/"$TRAIN_PROBLEM_NAME" \
    --eval-problems-paths ../data/"$PROBLEM_TYPE"/val/"$VAL_PROBLEM_NAME" \
    --lr 0.00002 \
    --bsize 64 \
    --buffer-size 20000 \
    --eps-init 1.0 \
    --eps-final 0.01 \
    --gamma 0.99 \
    --eps-decay-steps 30000 \
    --batch-updates 50000 \
    --history-len 1 \
    --init-exploration-steps 5000 \
    --step-freq 4 \
    --target-update-freq 10 \
    --loss mse \
    --opt adam \
    --save-freq 500 \
    --grad_clip 0.1 \
    --grad_clip_norm_type 2 \
    --eval-freq 1000 \
    --eval-time-limit 3600 \
    --core-steps 4 \
    --expert-exploration-prob 0.0 \
    --priority_alpha 0.5 \
    --priority_beta 0.5 \
    --e2v-aggregator sum \
    --n_hidden 1 \
    --hidden_size 64 \
    --decoder_v_out_size 32 \
    --decoder_e_out_size 1 \
    --decoder_g_out_size 1 \
    --encoder_v_out_size 32 \
    --encoder_e_out_size 32 \
    --encoder_g_out_size 32 \
    --core_v_out_size 64 \
    --core_e_out_size 64 \
    --core_g_out_size 32 \
    --activation relu \
    --penalty_size 0.1 \
    --train_time_max_decisions_allowed 500 \
    --test_time_max_decisions_allowed 500 \
    --no_max_cap_fill_buffer \
    --lr_scheduler_gamma 1 \
    --lr_scheduler_frequency 3000 \
    --independent_block_layers 0

### Add metadata for evaluation (test set)

In [ ]:
for PROBLEM_NAME in datasets[PROBLEM_TYPE]['inner_test']:
  !python add_metadata.py --eval-problems-paths ../data/"$PROBLEM_TYPE"/test/"$PROBLEM_NAME"

for PROBLEM_NAME in datasets[PROBLEM_TYPE]['test']:
  !python add_metadata.py --eval-problems-paths ../data/"$PROBLEM_TYPE"/"$PROBLEM_NAME"

### Evaluate

In [ ]:
MODEL_DIR='Sep10_12-34-08_19c1d7476815'
CHECKPOINT='model_50003'

In [ ]:
for PROBLEM_NAME in datasets[PROBLEM_TYPE]['test']:
  for MODEL_DECISION in [10, 50, 100, 300, 500, 1000]:
    !python evaluate.py \
      --logdir log \
      --env-name sat-v0 \
      --core-steps -1 \
      --eps-final 0.0 \
      --eval-time-limit 100000000000000 \
      --no_restarts \
      --test_time_max_decisions_allowed "$MODEL_DECISION" \
      --eval-problems-paths ../data/"$PROBLEM_TYPE"/"$PROBLEM_NAME" \
      --model-dir runs/"$MODEL_DIR" \
      --model-checkpoint "$CHECKPOINT".chkp \
      >> runs/"$MODEL_DIR"/"$PROBLEM_NAME"-graphqsat-max"$MODEL_DECISION".tsv

In [ ]:
%cd ..

/content/gdrive/My Drive/neuro-sat
